In [35]:
%pip install pandas
%pip install torch
%pip install scikit-learn
%pip install numpy
%pip install dask
%pip install pyarrow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\mannat2\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\mannat2\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\mannat2\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\mannat2\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\mannat2\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# Goal: Build a classification model to segment the customers based on the account data.

classify the Accounts eligible for Credit Line Increase without Risk, Accounts eligible for Credit Line
Increase but has risk of potential defaults or fraud, No Credit Line Increase required and Non-Performing
accounts that pose a high risk.

# Data Cleaning and Preprocessing

In [36]:
import pandas as pd
import dask.dataframe as dd
import pyarrow as pa

In [37]:
account_dim = pd.read_csv("C:/Users/mannat2/datathon/consumer_spending_ml/data/tmp/datathon/data/account_dim_20250325.csv")
fraud_claim_case = pd.read_csv("C:/Users/mannat2/datathon/consumer_spending_ml/data/tmp/datathon/data/fraud_claim_case_20250325.csv")
fraud_claim_tran = pd.read_csv("C:/Users/mannat2/datathon/consumer_spending_ml/data/tmp/datathon/data/fraud_claim_tran_20250325.csv")
rams_batch_cur = pd.read_csv("C:/Users/mannat2/Documents/GitHub/consumer_spending_ml/data/tmp/datathon/data/rams_batch_cur_20250325.csv")
statement_fact = pd.read_csv("C:/Users/mannat2/Documents/GitHub/consumer_spending_ml/data/tmp/datathon/data/statement_fact_20250325.csv")
syf_id = pd.read_csv("C:/Users/mannat2/Documents/GitHub/consumer_spending_ml/data/tmp/datathon/data/syf_id_20250325.csv")
transaction_fact = pd.read_csv("C:/Users/mannat2/Documents/GitHub/consumer_spending_ml/data/tmp/datathon/data/transaction_fact_20250325.csv")
wrld_stor_tran_fact = pd.read_csv("C:/Users/mannat2/Documents/GitHub/consumer_spending_ml/data/tmp/datathon/data/wrld_stor_tran_fact_20250325.csv")

In [38]:
rams_batch_cur = rams_batch_cur[['cu_account_nbr', 'cu_bhv_scr', 'cu_crd_bureau_scr', 'ca_avg_utilz_lst_6_mnths', 'ca_avg_utilz_lst_6_mnths', 'mo_tot_sales_array_1', 'mo_tot_sales_array_2', 'mo_tot_sales_array_3', 'mo_tot_sales_array_4', 'mo_tot_sales_array_5', 'mo_tot_sales_array_6', 'cu_cash_line_am', 'cu_crd_line', 'cu_otb', 'cu_nbr_days_dlq', 'ca_max_dlq_lst_6_mnths', 'ca_nsf_count_lst_12_months']]
account_dim = account_dim[['current_account_nbr', 'payment_hist_1_12_mths', 'payment_hist_13_24_mths', 'overlimit_type_flag', 'external_status_reason_code']]
statement_fact = statement_fact[['current_account_nbr', 'return_check_cnt_total']]
fraud_claim_case = fraud_claim_case[['current_account_nbr', 'gross_fraud_amt', 'net_fraud_amt']]
#transaction_fact = transaction_fact[['current_account_nbr', 'transaction_return_cnt', 'first_purchase_ind']]
#transaction_fact = dd.from_pandas(transaction_fact, npartitions=10)

In [39]:
rams_batch_cur = rams_batch_cur.rename(columns={"cu_account_nbr": "current_account_nbr"})
fraud_claim_case = fraud_claim_case.rename(columns={"current_account_nbr_pty": "current_account_nbr"})

In [40]:
master_df = account_dim.merge(rams_batch_cur, on='current_account_nbr', how='outer')
aster_df = master_df.merge(statement_fact, on="current_account_nbr", how='outer')
master_df = master_df.merge(fraud_claim_case, on="current_account_nbr", how='outer')

In [41]:
master_df.drop(columns=['overlimit_type_flag', 'ca_avg_utilz_lst_6_mnths', 'payment_hist_1_12_mths', 'payment_hist_13_24_mths'], inplace=True)

In [42]:
# check if this is the right way to do and deal with the NAN values
master_df.fillna(0, inplace=True)

# K-Means Clustering

In [67]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [68]:
master_df.isna().value_counts()

current_account_nbr  external_status_reason_code  cu_bhv_scr  cu_crd_bureau_scr  mo_tot_sales_array_1  mo_tot_sales_array_2  mo_tot_sales_array_3  mo_tot_sales_array_4  mo_tot_sales_array_5  mo_tot_sales_array_6  cu_cash_line_am  cu_crd_line  cu_otb  cu_nbr_days_dlq  ca_max_dlq_lst_6_mnths  ca_nsf_count_lst_12_months  gross_fraud_amt  net_fraud_amt  cluster  customer_segmentation
False                False                        False       False              False                 False                 False                 False                 False                 False                 False            False        False   False            False                   False                       False            False          False    False                    18070
Name: count, dtype: int64

In [69]:
master_df = master_df.groupby('current_account_nbr').agg('mean').reset_index()

TypeError: agg function failed [how->mean,dtype->object]

In [ ]:
master_df

,current_account_nbr,external_status_reason_code,cu_bhv_scr,cu_crd_bureau_scr,mo_tot_sales_array_1,mo_tot_sales_array_2,mo_tot_sales_array_3,mo_tot_sales_array_4,mo_tot_sales_array_5,mo_tot_sales_array_6,cu_cash_line_am,cu_crd_line,cu_otb,cu_nbr_days_dlq,ca_max_dlq_lst_6_mnths,ca_nsf_count_lst_12_months,gross_fraud_amt,net_fraud_amt
0,00gwujJkd597VxLd,0.0,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,00iP5U82D8XwVQ9G,0.0,738.800000,790.000000,535.768000,557.180000,540.090,442.250000,349.814000,294.038000,0.000000e+00,3500.0,2966.928000,0.0,0.0,0.0,0.0,0.0
2,00oyr3QppAzjLws4,0.0,662.833333,664.000000,40.803333,40.803333,60.805,70.613333,54.110000,85.585000,0.000000e+00,2900.0,287.896667,0.0,0.0,0.0,0.0,0.0
3,01SEhQXHbPJRc1Go,35.0,12.000000,729.833333,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,1.000000e+15,1750.0,1750.000000,0.0,0.0,0.0,0.0,0.0
4,01wbqz3obYPYxLvR,0.0,5.750000,623.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,1.000000e+15,400.0,400.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18065,zzBy2qNM78aRV580,46.0,737.500000,795.833333,0.000000,0.000000,0.000,419.436667,476.101667,476.101667,2.960000e+03,14800.0,14487.775000,0.0,0.0,0.0,0.0,0.0
18066,zzEuUBBmvGiVnabb,0.0,746.833333,758.000000,1920.125000,1793.153333,1478.100,1245.646667,1001.271667,1102.381667,1.200000e+03,6000.0,5060.371667,0.0,0.0,0.0,0.0,0.0
18067,zzR9PvG7dY9u5iHU,0.0,3.000000,650.000000,471.690000,90.300000,0.000,0.000000,0.000000,0.000000,1.000000e+03,5000.0,4514.510000,0.0,0.0,0.0,0.0,0.0
18068,zzXGgGu6ysGwGH1J,0.0,0.000000,653.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,1.000000e+15,1000.0,1000.000000,0.0,0.0,0.0,0.0,0.0


In [70]:
standardized_columns = [
    'external_status_reason_code', 'cu_bhv_scr', 'cu_crd_bureau_scr', 'mo_tot_sales_array_1', 
    'mo_tot_sales_array_2', 'mo_tot_sales_array_3', 'mo_tot_sales_array_4', 'mo_tot_sales_array_5', 
    'mo_tot_sales_array_6', 'cu_cash_line_am', 'cu_crd_line', 'cu_otb', 'cu_nbr_days_dlq', 'ca_max_dlq_lst_6_mnths', 
    'ca_nsf_count_lst_12_months', 'gross_fraud_amt', 'net_fraud_amt'
]

In [71]:
scaler = StandardScaler()

In [72]:
master_df_scaled = scaler.fit_transform(master_df[standardized_columns])
master_df_scaled = pd.DataFrame(master_df_scaled, columns=standardized_columns)

In [73]:
master_df[standardized_columns] = master_df_scaled

In [74]:
master_df

,current_account_nbr,external_status_reason_code,cu_bhv_scr,cu_crd_bureau_scr,mo_tot_sales_array_1,mo_tot_sales_array_2,mo_tot_sales_array_3,mo_tot_sales_array_4,mo_tot_sales_array_5,mo_tot_sales_array_6,cu_cash_line_am,cu_crd_line,cu_otb,cu_nbr_days_dlq,ca_max_dlq_lst_6_mnths,ca_nsf_count_lst_12_months,gross_fraud_amt,net_fraud_amt,cluster,customer_segmentation
0,00gwujJkd597VxLd,-0.384367,-1.368876,-4.164438,-0.417710,-0.411155,-0.402959,-0.400066,-0.400409,-0.404253,-0.707342,-1.036076,-0.949995,-0.23563,-0.313281,-0.106926,-0.033485,-0.023347,0,Accounts eligible for Credit Line Increase wit...
1,00iP5U82D8XwVQ9G,-0.384367,0.933246,0.449975,-0.015433,0.019050,0.004483,-0.065552,-0.132237,-0.179962,-0.707342,-0.457709,-0.405125,-0.23563,-0.313281,-0.106926,-0.033485,-0.023347,0,Accounts eligible for Credit Line Increase wit...
2,00oyr3QppAzjLws4,-0.384367,0.696531,-0.285995,-0.387074,-0.379651,-0.357088,-0.346654,-0.358928,-0.338969,-0.707342,-0.556858,-0.897124,-0.23563,-0.313281,-0.106926,-0.033485,-0.023347,0,Accounts eligible for Credit Line Increase wit...
3,01SEhQXHbPJRc1Go,1.906133,-1.331483,0.098540,-0.417710,-0.411155,-0.402959,-0.400066,-0.400409,-0.404253,1.413744,-0.746893,-0.628611,-0.23563,-0.313281,-0.106926,-0.033485,-0.023347,0,Accounts eligible for Credit Line Increase wit...
4,01wbqz3obYPYxLvR,-0.384367,-1.350959,-0.525477,-0.417710,-0.411155,-0.402959,-0.400066,-0.400409,-0.404253,1.413744,-0.969977,-0.876536,-0.23563,-0.313281,-0.106926,-0.033485,-0.023347,0,Accounts eligible for Credit Line Increase wit...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18065,zzBy2qNM78aRV580,2.626005,0.929195,0.484048,-0.417710,-0.411155,-0.402959,-0.082808,-0.035423,-0.041084,-0.707342,1.409591,1.710657,-0.23563,-0.313281,-0.106926,-0.033485,-0.023347,0,Accounts eligible for Credit Line Increase wit...
18066,zzEuUBBmvGiVnabb,-0.384367,0.958278,0.263062,1.024000,0.973361,0.712115,0.542128,0.367179,0.436640,-0.707342,-0.044589,-0.020668,-0.23563,-0.313281,-0.106926,-0.033485,-0.023347,0,Accounts eligible for Credit Line Increase wit...
18067,zzR9PvG7dY9u5iHU,-0.384367,-1.359528,-0.367769,-0.063546,-0.341434,-0.402959,-0.400066,-0.400409,-0.404253,-0.707342,-0.209837,-0.120914,-0.23563,-0.313281,-0.106926,-0.033485,-0.023347,0,Accounts eligible for Credit Line Increase wit...
18068,zzXGgGu6ysGwGH1J,-0.384367,-1.368876,-0.350246,-0.417710,-0.411155,-0.402959,-0.400066,-0.400409,-0.404253,1.413744,-0.870828,-0.766347,-0.23563,-0.313281,-0.106926,-0.033485,-0.023347,0,Accounts eligible for Credit Line Increase wit...


In [75]:
kmeans = KMeans(n_clusters=4, random_state=42)
kmeans.fit(master_df[standardized_columns])

KMeans(n_clusters=4, random_state=42)

In [76]:
score = silhouette_score(master_df[standardized_columns], kmeans.labels_, metric='euclidean')
score

np.float64(0.3878650561007108)

In [77]:
predictions = kmeans.predict(master_df[standardized_columns])

In [78]:
master_df['cluster'] = predictions

In [79]:
def cluster_summary(row):
    if (row['cluster'] == 0):
        return 'Accounts eligible for Credit Line Increase without Risk'
    elif (row['cluster'] == 1):
        return 'Non-performing accounts that pose a high risk'
    elif (row['cluster'] == 2):
        return 'Accounts eligible for Credit Line Increase but has risk of potential defaults or fraud'
    else:
        return 'No Credit Line Increase required'

In [80]:
master_df['customer_segmentation'] = master_df.apply(cluster_summary, axis=1)

In [81]:
master_df['customer_segmentation'].value_counts()

customer_segmentation
Accounts eligible for Credit Line Increase but has risk of potential defaults or fraud    13660
Non-performing accounts that pose a high risk                                              2360
Accounts eligible for Credit Line Increase without Risk                                    1321
No Credit Line Increase required                                                            729
Name: count, dtype: int64

# Results

Cluster 0: Accounts eligible for Credit Line Increase without Risk

Cluster 1: Non-performing accounts that pose a high risk

Cluster 2: Accounts eligible for Credit Line Increase but has risk of potential defaults or fraud 

Cluster 3: No Credit Line Increase required

Current silhouette score: 0.2

# PCA Analysis

In [ ]:
import numpy as np
from sklearn.decomposition import PCA

pca = PCA(n_components=4)
pca.fit(master_df[standardized_columns])

# Next Steps

Check with PCA and apiori algorithm

-- try to incorporate above to increase the silhouette score and make more accurate clusters/higher score